In [37]:
import sys
sys.path.append(r'/home/nkrokhmal/Desktop/MyProjects/SergeyProject/umalat-project/prod')
from app.models import *
from app.models import Boiling as BoilingModel
from app.models import Termizator as TermizatorModel
from openpyxl.utils.cell import coordinate_from_string, column_index_from_string


from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

# sqlite_filepath = '/home/akadaner/Desktop/data.sqlite'
sqlite_filepath = r"/home/nkrokhmal/Desktop/MyProjects/SergeyProject/umalat-project/prod/data.sqlite"
engine = create_engine(f"sqlite:///{sqlite_filepath}")
Session = sessionmaker()
Session.configure(bind=engine)
session = Session()

def cast_sku(obj):
    if isinstance(obj, SKU):
        return obj
    elif isinstance(obj, (str, int)):
        obj = str(obj)
        return session.query(SKU).filter(SKU.id == obj).first()
    else:
        raise Exception('Unknown sku type')
        
def cast_boiling(obj):
    if isinstance(obj, Boiling):
        return obj
    elif isinstance(obj, (str, int)):
        obj = str(obj)
        return session.query(Boiling).filter(Boiling.id == obj).first()
    else:
        raise Exception('Unknown boiling type')

In [3]:
lst = ['gfg', 'is', 'a', 'portal', 'for', 'geeks'] 
  
# Using sort() function 
sorted(lst)

In [2]:
lst

In [6]:
datetime.datetime.strptime('01.01.2020', '%d.%m.%Y')
[{"BoilingId": 6, "BoilingCount": 1.0, "BoilingWeights": [8000], 
  "SKUVolumes": {"24": 199, "23": 190.3999999999999, "26": 75.6, "27": 40.5, "22": 76.5}}

In [36]:
import re
# a = '=-(F21+F22)/K25'.replace("(", "").replace(")", "")
a = '=-(F21+F22)/K25'
tmp = re.split('=|-|/|\(|\)|\+', a)
tmp

In [38]:
def get_index(coordinate):
    xy = coordinate_from_string(coordinate) # returns ('A',4)
    col = column_index_from_string(xy[0]) # returns 1
    row = xy[1]
    return row, col

In [76]:
def parse_json(path):
    basename = os.path.basename(path)
    date_str = os.path.splitext(basename)[0]
    date = datetime.datetime.strptime(date_str, '%d.%m.%y')

    response = {'Date': date, 'WeekDay': date.weekday(), 'Boilings': []}
    
    skus = session.query(SKU).all()
    wb = openpyxl.load_workbook(filename=path)
    wb_data = openpyxl.load_workbook(filename=path, data_only=True)
    ws = wb['планирование суточное']
    ws_data = wb_data['планирование суточное']
    for i in range(1, 200):
        cell = ws.cell(i, 11).value
        if cell is not None:
            splited = re.split('=|-|/|\(|\)|\+|:', str(cell))
            splited = [get_index(x) for x in splited if x.startswith('F')]
            
            if len(splited) > 0:
                sku_names = [ws_data.cell(x[0], 3).value for x  in splited]
                boilings = [x.boilings[0] for x in skus if x.name in sku_names]
                if len(boilings) > 0:
                    if ws_data.cell(i, 12).value is not None:
                        tmp = {}
                        tmp['BoilingId'] = boilings[0].id
                        tmp['BoilingCount'] = ws_data.cell(i, 12).value
                        tmp['BoilingWeights'] = int(tmp['BoilingCount']) * [8000]
                        tmp_list = []
                        for s in splited:
                            sku_name = ws_data.cell(s[0], 3).value
                            sku_volume = ws_data.cell(s[0], 6).value
                            sku_ids = [x.id for x in skus if x.name == sku_name]

                            if len(sku_ids) > 0 and sku_volume is not None and sku_volume < 0:
                                tmp_list.append({sku_ids[0]: - sku_volume})
                        tmp['SKUVolumes'] = tmp_list
                        response['Boilings'].append(tmp)
    return response
                    
#                     print(sku_name, sku_volume)

        

In [77]:
parse_json('/home/nkrokhmal/Desktop/MyProjects/SergeyProject/umalat-project/prod/research/nkrokhmal/30.09.20.xlsx')

In [40]:
def create_json(path):
    try:
        basename = os.path.basename(path)
        date_str = os.path.splitext(basename)[0]
        date = datetime.datetime.strptime(date_str, '%d.%m.%y')

        response = {'Date': date, 'WeekDay': date.weekday(), 'Boilings': []}
        result_list = []
        skus = session.query(SKU).all()
        group_items = [{
            "Ferment": x.boiling.ferment,
            "IsLactose": x.boiling.is_lactose,
            "Percent": x.boiling.percent,
            "FormFactor": x.form_factor
        } for x in skus.copy()]
        group_items = [dict(x) for x in set(frozenset(d.items()) for d in group_items)]

        df = pd.read_excel(path, index_col=0)
        df_save = df.copy()
        df.columns = range(df.shape[1])
        df = df[df.loc['Дата выработки продукции:'].dropna().index]
        df = df.loc[['Дата выработки продукции:',
                     'Заявлено всего, кг:',
                     'Фактические остатки на складах - Заявлено, кг:',
                     'Нормативные остатки, кг']].fillna(0).iloc[:, :-1]
        data = list(zip(*df.values.tolist()))

        full_list = []
        print('Creating full list')
        sku_for_create = []
        for item in data:
            sku = [x for x in skus if x.name == item[0]]
            if sku is not None and len(sku) > 0:
                full_list.append({
                    "SKU": sku[0],
                    "Request": item[2]
                })
            else:
                sku_for_create.append(item[0])
        print('Group items')
        for group_item in group_items:
            group_sku = [x for x in full_list if
                                 x["SKU"].boiling.ferment == group_item["Ferment"] and
                                 x["SKU"].boiling.is_lactose == group_item["IsLactose"] and
                                 x["SKU"].boiling.percent == group_item["Percent"] and
                                 x["SKU"].form_factor == group_item["FormFactor"]]
            if group_sku is not None and len(group_sku) > 0:
                output_weight = group_sku[0]["SKU"].output_kg
                request_weight = sum([x["Request"] for x in group_sku if x["Request"] < 0])
                result_list.append({
                    "GroupSKU": group_sku,
                    "BoilingId": group_sku[0]["SKU"].boiling_id,
                    "BoilingCount": - request_weight / output_weight
                })
        result_list = [x for x in result_list if round(x['BoilingCount']) > 0]
        print(len(result_list))
        for res in result_list:
            tmp = {}
            tmp['BoilingId'] = res['BoilingId']
            tmp['BoilingCount'] = round(res['BoilingCount'])
            tmp['BoilingWeights'] = round(res['BoilingCount']) * [8000]
            tmp_list = []
            for sku in res['GroupSKU']:
                if sku["Request"] < 0:
                    tmp_list.append({sku["SKU"].id: sku["Request"]})
            tmp['SKUVolumes'] = tmp_list
            response['Boilings'].append(tmp)
            
        saved_path = os.path.join(os.path.dirname(path), date_str + '.txt')
        print(saved_path)
        with open(saved_path, 'w') as outfile:
            def default(o):
                if isinstance(o, (datetime.date, datetime.datetime)):
                    return o.isoformat()
            json.dump(response, outfile, default=default)
        
#         return response
    except Exception as e:
        print('Exception occured {}'.format(e))

    

In [41]:
create_json('/home/nkrokhmal/Desktop/MyProjects/SergeyProject/umalat-project/prod/research/nkrokhmal/30.09.20.xlsx')

In [42]:
directory = '/home/nkrokhmal/Downloads/2020 scheduling/2020'
t = os.walk(directory)
for folder in [x[0] for x in os.walk(directory)][1:]:
    filenames = [x[2] for x in os.walk(folder)][0]
    for filename in filenames:
        filename_fullpath = os.path.join(folder, filename)
        print(filename_fullpath)
        create_json(filename_fullpath)
    

In [25]:
[x[0] for x in os.walk(directory)]

In [21]:
[x[2] for x in os.walk('/home/nkrokhmal/Downloads/2020 scheduling/2020/Июль')]